1. Визначити номер варіанта за формулою:

In [6]:
FIRST_LETTER_OF_MY_NAME = "M"

print("Mій варіант:", ord(FIRST_LETTER_OF_MY_NAME) % 5 + 1)

Mій варіант: 3


In [7]:
import pandas as pd

df = pd.read_excel("lab5.xlsx")

class ImageData:          
    def __init__(self, data: pd.Series):
        self.filename = data["file name"]
        self.image_sizes =[int(data) for data in data["image size"].split("x")]
        self.glasses_color = data["glasses color"]
        self.line_width = int(data["line width"])
        
    def __str__(self):
        return f"filename: {self.filename}, image size: {self.image_sizes}, glasses color: {self.glasses_color}, line width: {self.line_width}"

image = ImageData(df[df["N"] == 3].iloc[0])

image.__str__()

'filename: draco.jpg, image size: [700, 700], glasses color: cyan, line width: 4'

In [8]:
import cv2
from PIL import Image, ImageDraw


class Eye:
    def __init__(self, x: int, y: int, w: int, h: int):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.center = (x + w // 2, y + h // 2)
        self.radius = min(w, h) // 2


def detect_face(image: Image):
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100)
    )

    if len(faces) == 0:
        raise Exception("No faces found")

    return sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]


def detect_eyes(image: Image):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detected_eyes = sorted(
        eye_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30)
        ),
        key=lambda eye: eye[0],
    )

    return [Eye(ex, ey, ew, eh) for (ex, ey, ew, eh) in detected_eyes]


def cut_image(image: Image, center, sizes):
    x, y = center
    w, h = sizes
    half_width, half_height = w // 2, h // 2
    x1, y1 = max(0, x - half_width), max(0, y - half_height)
    x2, y2 = min(image.shape[1], x + half_width), min(image.shape[0], y + half_height)

    return image[y1:y2, x1:x2]


def draw_glasses(image: Image, eyes: list[Eye], color: str, line_width: int):
    result_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    result_draw = ImageDraw.Draw(result_image)

    for eye in eyes:
        result_draw.ellipse(
            [
                (eye.center[0] - eye.radius, eye.center[1] - eye.radius),
                (eye.center[0] + eye.radius, eye.center[1] + eye.radius),
            ],
            outline=color,
            width=line_width,
        )

    if(len(eyes) == 2):
        result_draw.line(
            [
                (eyes[0].center[0] + eyes[0].radius, eyes[0].center[1]),
                (eyes[1].center[0] - eyes[1].radius, eyes[1].center[1]),
            ],
            fill=color,
            width=line_width,
        )

    return result_image


def process_image(path: str, data: ImageData, output_path: str = None) -> str:
    if output_path is None:
        output_path = f"{path}/result_{data.filename}"
    image = cv2.imread(f"{path}/{data.filename}")

    (x, y, w, h) = detect_face(image)
    image = cut_image(image, (x + w // 2, y + h // 2), data.image_sizes)

    eyes = detect_eyes(image)
    result_image = draw_glasses(image, eyes, data.glasses_color, data.line_width)
    result_image.save(output_path)

    return output_path


process_image("./Images", image)

'./Images/result_draco.jpg'